## 0. Preparación del notebook

In [1]:
import os
import pandas as pd
import json
import textwrap
from scipy import spatial
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv

from IPython.display import display # Sólo para la ejecución en Jupyter

load_dotenv("../../../../../../../apis/.env")
api_key = os.getenv("OPENAI_API_KEY")
unmasked_chars = 8
masked_key = api_key[:unmasked_chars] + '*' * (len(api_key) - unmasked_chars*2) + api_key[-unmasked_chars:]
print(f"API key: {masked_key}")

API key: sk-proj-****************************************************************************************************************************************************-amA_5sA


## 1. Funciones de procesamiento de datos y cálculo de puntuación

In [ ]:
class ProcesadorCV:

    def __init__(self, api_key, cv_text, job_text, ner_pre_prompt, ner_schema,
                inference_model="gpt-4o-mini", embeddings_model="text-embedding-3-small"):
        """
        Inicializa una instancia de la clase con los parámetros proporcionados.

        Args:
            api_key (str): La clave de API para autenticar con el cliente OpenAI.
            cv_text (str): contenido del CV en formato de texto.
            job_text (str): título de la oferta de trabajo a evaluar.
            ner_pre_prompt (str): instrucción de "reconocimiento de entidades nombradas" (NER) para el modelo en lenguaje natural.
            ner_schema (dict): esquema para la llamada con "structured outputs" al modelo de OpenAI.
            inference_model (str, opcional): El modelo de inferencia a utilizar. Por defecto es "gpt-4o-mini".
            embeddings_model (str, opcional): El modelo de embeddings a utilizar. Por defecto es "text-embedding-3-small".

        Atributos:
            inference_model (str): Almacena el modelo de inferencia seleccionado.
            embeddings_model (str): Almacena el modelo de embeddings seleccionado.
            client (OpenAI): Instancia del cliente OpenAI inicializada con la clave de API proporcionada.
            cv (str): Almacena el texto del currículum vitae proporcionado.

        """
        self.inference_model = inference_model
        self.embeddings_model = embeddings_model
        self.ner_pre_prompt = ner_pre_prompt
        self.ner_schema = ner_schema
        self.client = OpenAI(api_key=api_key)
        self.cv = cv_text
        self.job_text = job_text
        print("Cliente inicializado como",self.client)

    def extraer_datos_cv(self, temperature=0.5):
        """
        Extrae datos estructurados de un CV con OpenAI API.
        Args:
            pre_prompt (str): instrucción para el modelo en lenguaje natural.
            schema (dict): esquema de los parámetros que se espera extraer del CV.
            temperature (float, optional): valor de temperatura para el modelo de lenguaje. Por defecto es 0.5.
        Returns:
            pd.DataFrame: DataFrame con los datos estructurados extraídos del CV.
        Raises:
            ValueError: si no se pueden extraer datos estructurados del CV.
        """
        response = self.client.chat.completions.create(
            model=self.inference_model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": self.ner_pre_prompt},
                {"role": "user", "content": self.cv}
            ],
            functions=[
                {
                    "name": "extraer_datos_cv",
                    "description": "Extrae tabla con títulos de puesto de trabajo, nombres de empresa y períodos de un CV.",
                    "parameters": self.ner_schema
                }
            ],
            function_call="auto"
        )

        if response.choices[0].message.function_call:
            function_call = response.choices[0].message.function_call
            structured_output = json.loads(function_call.arguments)
            if structured_output.get("experiencia"):
                df_cv = pd.DataFrame(structured_output["experiencia"]) 
                return df_cv
            else:
                raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
        else:
            raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
        

    def procesar_periodos(self, df):    
        """
        Procesa los períodos en un DataFrame y añade columnas con las fechas de inicio, fin y duración en meses. 
        Si no hay fecha de fin, se considera la fecha actual.
        Args:
            df (pandas.DataFrame): DataFrame que contiene una columna 'periodo' con períodos en formato 'YYYYMM-YYYYMM' o 'YYYYMM'.
        Returns:
            pandas.DataFrame: DataFrame con columnas adicionales 'fec_inicio', 'fec_final' y 'duracion'.
                - 'fec_inicio' (datetime.date): Fecha de inicio del período.
                - 'fec_final' (datetime.date): Fecha de fin del período.
                - 'duracion' (int): Duración del período en meses.
        """
        # Función lambda para procesar el período
        def split_periodo(periodo):
            dates = periodo.split('-')
            start_date = datetime.strptime(dates[0], "%Y%m")
            if len(dates) > 1:
                end_date = datetime.strptime(dates[1], "%Y%m")
            else:
                end_date = datetime.now()
            return start_date, end_date

        df[['fec_inicio', 'fec_final']] = df['periodo'].apply(lambda x: pd.Series(split_periodo(x)))

        # Formateamos las fechas para mostrar mes, año, y el primer día del mes (dado que el día es irrelevante y no se suele especificar)
        df['fec_inicio'] = df['fec_inicio'].dt.date
        df['fec_final'] = df['fec_final'].dt.date

        # Añadimos una columna con la duración en meses
        df['duracion'] = df.apply(
            lambda row: (row['fec_final'].year - row['fec_inicio'].year) * 12 + 
                        row['fec_final'].month - row['fec_inicio'].month, 
            axis=1
        )

        return df


    def calcular_embeddings(self, df, column='puesto', model_name='text-embedding-3-small'):
        """
        Calcula los embeddings de una columna de un dataframe con OpenAI API.
        Args:
            cv_df (pandas.DataFrame): DataFrame con los datos de los CV.
            column (str, optional): Nombre de la columna que contiene los datos a convertir en embeddings. Por defecto es 'puesto'.
            model_name (str, optional): Nombre del modelo de embeddings. Por defecto es 'text-embedding-3-small'.
        """
        df['embeddings'] = df[column].apply(
            lambda puesto: self.client.embeddings.create(
                input=puesto, 
                model=model_name
            ).data[0].embedding
        )
        return df


    def calcular_distancias(self, df, column='embeddings', model_name='text-embedding-3-small'):
        """
        Calcula la distancia coseno entre los embeddings del texto y los incluidos en una columna del dataframe.
        Params:
        df (pandas.DataFrame): DataFrame que contiene los embeddings.
        column (str, optional): nombre de la columna del DataFrame que contiene los embeddings. Por defecto, 'embeddings'.
        model_name (str, optional): modelo de embeddings de la API de OpenAI. Por defecto "text-embedding-3-small".
        Returns:
        pandas.DataFrame: DataFrame ordenado de menor a mayor distancia, con las distancias en una nueva columna.
        """
        response = self.client.embeddings.create(
            input=self.job_text,
            model=model_name
        )
        emb_compare = response.data[0].embedding

        df['distancia'] = df[column].apply(lambda emb: spatial.distance.cosine(emb, emb_compare))
        df.drop(columns=[column], inplace=True)
        df.sort_values(by='distancia', ascending=True, inplace=True)
        return df


    def calcular_puntuacion(self, df, req_experience, positions_cap=4, dist_threshold_low=0.6, dist_threshold_high=0.7):
        """
        Calcula la puntuación de un CV a partir de su tabla de distancias (con respecto a un puesto dado) y duraciones. 

        Params:
        df (pandas.DataFrame): datos de un CV incluyendo diferentes experiencias incluyendo duracies y distancia previamente calculadas sobre los embeddings de un puesto de trabajo
        req_experience (float): experiencia requerida en meses para el puesto de trabajo (valor de referencia para calcular una puntuación entre 0 y 100 en base a diferentes experiencias)
        positions_cap (int, optional): Maximum number of positions to consider for scoring. Defaults to 4.
        dist_threshold_low (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV se considera "equivalente" al de la oferta.
        max_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV no puntúa.
        
        Returns:
        pandas.DataFrame: DataFrame original añadiendo una columna con las puntuaciones individuales contribuidas por cada puesto.
        float: Puntuación total entre 0 y 100.
        """
        # A efectos de puntuación, computamos para cada puesto como máximo el número total de meses de experiencia requeridos
        df['duration_capped'] = df['duracion'].apply(lambda x: min(x, req_experience))
        # Normalizamos la distancia entre 0 y 1, siendo 0 la distancia mínima y 1 la máxima
        df['adjusted_distance'] = df['distancia'].apply(
            lambda x: 0 if x <= dist_threshold_low else (
                1 if x >= dist_threshold_high else (x - dist_threshold_low) / (dist_threshold_high - dist_threshold_low)
            )
        )
        # Cada puesto puntúa en base a su duración y a la inversa de la distancia (a menor distancia, mayor puntuación)
        df['position_score'] = round(((1 - df['adjusted_distance']) * (df['duration_capped']/req_experience) * 100), 2)
        # Descartamos puestos con distancia superior al umbral definido (asignamos puntuación 0), y ordenamos por puntuación
        df.loc[df['distancia'] >= dist_threshold_high, 'position_score'] = 0
        df = df.sort_values(by='position_score', ascending=False)
        # Nos quedamos con los puestos con mayor puntuación (positions_cap)
        df.iloc[positions_cap:, df.columns.get_loc('position_score')] = 0
        # Totalizamos (no debería superar 100 nunca, pero ponemos un límite para asegurar) y redondeamos a dos decimales
        total_score = round(min(df['position_score'].sum(), 100), 2)
        return df, total_score
    
    def filtra_experiencia_relevante(self, df):
        """
        Filtra las experiencias relevantes del dataframe y las devuelve en formato diccionario.
        Args:
            df (pandas.DataFrame): DataFrame con la información completa de experiencia.
        Returns:
            dict: Diccionario con las experiencias relevantes.
        """
        df_experiencia =  df[df['position_score'] > 0].copy()
        df_experiencia.drop(columns=['periodo', 'fec_inicio', 'fec_final', 
                                     'distancia', 'duration_capped', 'adjusted_distance'], inplace=True)
        experiencia_dict = df_experiencia.to_dict(orient='list')
        return experiencia_dict
    
    def procesar_cv_completo(self, req_experience, positions_cap, dist_threshold_low, dist_threshold_high):
        """
        Procesa un CV y calcula la puntuación final.
        Args:
            req_experience (int, optional): Experiencia requerida en meses para el puesto de trabajo.
            positions_cap (int, optional): Número máximo de puestos a considerar para la puntuación.
            dist_threshold_low (float, optional): Distancia límite para considerar un puesto equivalente.
            dist_threshold_high (float, optional): Distancia límite para considerar un puesto no relevante.
        Returns:
            pd.DataFrame: DataFrame con las puntuaciones individuales contribuidas por cada puesto.
            float: Puntuación total entre 0 y 100.
        """
        df_datos_estructurados_cv = self.extraer_datos_cv()
        df_datos_estructurados_cv = self.procesar_periodos(df_datos_estructurados_cv)
        df_con_embeddings = self.calcular_embeddings(df_datos_estructurados_cv)
        df_con_distancias = self.calcular_distancias(df_con_embeddings)
        df_puntuaciones, puntuacion = self.calcular_puntuacion(df_con_distancias,
                                                                req_experience=req_experience,
                                                                positions_cap=positions_cap,
                                                                dist_threshold_low=dist_threshold_low,
                                                                dist_threshold_high=dist_threshold_high)
        dict_experiencia = self.filtra_experiencia_relevante(df_puntuaciones)
        return df_puntuaciones, puntuacion, dict_experiencia

### 2. Proceso completo de cálculo de puntuación

En el siguiente bloque, podemos introducir cualquier texto de oferta, un CV, y obtener las puntuaciones y el DataFrame con los cálculos:

In [ ]:
# Definimos la oferta de trabajo:
job_text = "Cajero supermercado Dia"

# Cargamos el esquema:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as schema_file:
    ner_schema = json.load(schema_file)

# Cargamos el CV:
cv_sample_path = '../../ejemplos_cvs/cv_sample.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()

# Cargamos el prompt para NER:
with open('../prompts/ner_pre_prompt.txt', 'r', encoding='utf-8') as file:
    ner_pre_prompt = file.read()

procesador_cvs = ProcesadorCV(api_key, cv_text, job_text, ner_pre_prompt, ner_schema)
req_experience = 12 # Experiencia requerida en meses
positions_cap=4 # Número máximo de puestos a considerar
dist_threshold_low=0.58 # Distancia límite para considerar un puesto equivalente
dist_threshold_high=0.62 # Distancia límite para considerar un puesto no relevante
df_puntuaciones, puntuacion, dict_experiencia = procesador_cvs.procesar_cv_completo(req_experience=req_experience,
                                                positions_cap=positions_cap,
                                                dist_threshold_low=dist_threshold_low,
                                                dist_threshold_high=dist_threshold_high
                                                )

print(f"Puntuación: {puntuacion:.1f}/100")
pd.options.display.float_format = '{:,.2f}'.format
display(df_puntuaciones)
pd.reset_option('display.float_format')
print(dict_experiencia)

Cliente inicializado como <openai.OpenAI object at 0x00000159FCE43C90>
Puntuación: 89.0/100


,empresa,puesto,periodo,fec_inicio,fec_final,duracion,distancia,duration_capped,adjusted_distance,position_score
1,Mercadona,Vendedor/a de puesto de mercado,202310-202403,2023-10-01,2024-03-01,5,0.56,5,0.00,41.67
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6,0.59,6,0.18,40.87
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48,0.62,12,0.94,6.47
0,Autónomo,Comercial de automoviles,202401-202402,2024-01-01,2024-02-01,1,0.63,1,1.00,0.00
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6,0.65,6,1.00,0.00
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29,0.70,12,1.00,0.00


{'empresa': ['Mercadona', 'GASTROTEKA ORDIZIA 1990', 'AGRISOLUTIONS'], 'puesto': ['Vendedor/a de puesto de mercado', 'Camarero/a de barra', 'AUXILIAR DE MANTENIMIENTO INDUSTRIAL'], 'duracion': [5, 6, 48], 'position_score': [41.67, 40.87, 6.47]}


### 3. Llamada final al modelo de lenguaje

In [35]:
client = OpenAI(api_key=api_key)
print("Cliente inicializado como",client)

Cliente inicializado como <openai.OpenAI object at 0x00000159FCC15250>


Definimos un esquema para la respuesta final:

In [36]:
response_schema = {
    "type": "object",
    "properties": {
        "puntuacion": {"type": "number"},
        "experiencia": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "empresa": {"type": "string"},
                    "puesto": {"type": "string"},
                    "duracion": {"type": "integer"}
                },
                "required": ["empresa", "puesto", "duracion"]
            }
        },
        "descripcion de la experiencia": {"type": "string"}
    },
    "required": ["puntuacion", "experiencia relevante", "descripcion de la experiencia"]
}

# Guardamos el esquema en un fichero JSON
with open('../json/response_schema.json', 'w', encoding='utf-8') as f:
    json.dump(response_schema, f, ensure_ascii=False, indent=4)

In [ ]:
# Recuperamos el esquema desde el fichero JSON guardado (para comprobar que funciona, ya que el código final utilizará el fichero)
with open('../json/response_schema.json', 'r', encoding='utf-8') as f:
    response_schema = json.load(f)

print(response_schema)

{'type': 'object', 'properties': {'puntuacion': {'type': 'number'}, 'experiencia': {'type': 'array', 'items': {'type': 'object', 'properties': {'empresa': {'type': 'string'}, 'puesto': {'type': 'string'}, 'duracion': {'type': 'integer'}}, 'required': ['empresa', 'puesto', 'duracion']}}, 'descripcion de la experiencia': {'type': 'string'}}, 'required': ['puntuacion', 'experiencia relevante', 'descripcion de la experiencia']}


Creamos un "system prompt" (instrucción general) y un "user prompt" (instrucción con contexto específico: puntuación y datos estructurados) para la inferencia final:

In [38]:
system_prompt = ("Eres un procesador de currículos vitae que recibe una oferta de trabajo un currículum vitae filtrado "
                "la experiencia relevante previa, una puntuación precalculada para el currículo entre 0 y 100, "
                "y un parámetro de experiencia requerida en meses. "
                "La puntuación se ha calculado mediante un algoritmo que usa distancias de embeddings entre cada uno de los puestos "
                "y la definición de la oferta, así como la duración de cada puesto y su relación con el parámetro de experiencia requerida. "
                "Devuelves un objeto con el esquema predefinido,"
                "incluyendo exactamente la misma puntuación proporcionada, el listado de experiencia proporcionado "
                "y además devuelves un breve texto explicativo sobre la experiencia del candidato y  "
                "por qué ha obtenido la puntuación dada. Es importante que el texto explicativo sea coherente con la puntuación. "
                "Por ejemplo, si la puntuación es mayor que 80, el texto explicativo debe hacer énfasis en las experiencias pasadas "
                "y la duración de las mismas que han llevado a esa puntuación. "
                "Cuando menciones algo en relación a la duración de la experiencia, asegúrate de convertirlo a años si es mayor que 12 meses."
            )

user_prompt = ("El título de la oferta de trabajo es: {job}." 
               "La experiencia requerida en meses es: {req_experience}." 
               "La puntuacion es {puntuacion}, "
               "La experiencia relevante es: {exp}. "
               "Explica por qué se ha obtenido la puntuación"
               )


# Los guardamos en ficheros de texto para simplificar el código y facilitar su mantenimiento y edición:
with open('../prompts/system_prompt.txt', 'w', encoding='utf-8') as f:
    f.write(system_prompt)

with open('../prompts/user_prompt.txt', 'w', encoding='utf-8') as f:
    f.write(user_prompt)

In [39]:
# Recuperamos los ficheros guardados para comprobar que están bien:
with open('../prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()

with open('../prompts/user_prompt.txt', 'r', encoding='utf-8') as f:
    user_prompt = f.read()

print("### System prompt ###")
print(textwrap.fill(system_prompt, width=120))
# En el caso del prompt del usuario, el texto contiene variables que serán reemplazadas por los valores correspondientes.
# Por ejemplo, usamos las definidas en este notebook para visualizar el texto que finalmente recibirá el modelo.
print("\n### User prompt ###")
user_prompt_con_contexto = user_prompt.format(job=job_text, req_experience=req_experience,puntuacion=puntuacion, exp=dict_experiencia)
print(textwrap.fill(user_prompt_con_contexto, width=120))

### System prompt ###
Eres un procesador de currículos vitae que recibe una oferta de trabajo un currículum vitae filtrado la experiencia
relevante previa, una puntuación precalculada para el currículo entre 0 y 100, y un parámetro de experiencia requerida
en meses. La puntuación se ha calculado mediante un algoritmo que usa distancias de embeddings entre cada uno de los
puestos y la definición de la oferta, así como la duración de cada puesto y su relación con el parámetro de experiencia
requerida. Devuelves un objeto con el esquema predefinido,incluyendo exactamente la misma puntuación proporcionada, el
listado de experiencia proporcionado y además devuelves un breve texto explicativo sobre la experiencia del candidato y
por qué ha obtenido la puntuación dada. Es importante que el texto explicativo sea coherente con la puntuación. Por
ejemplo, si la puntuación es mayor que 80, el texto explicativo debe hacer énfasis en las experiencias pasadas y la
duración de las mismas que han llev

In [ ]:
messages = [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": user_prompt.format(job=job_text, req_experience=req_experience,puntuacion=puntuacion, exp=dict_experiencia)
    }
]

functions = [
    {
        "name": "respuesta_formateada",
        "description": "Devuelve el objeto con puntuacion, experiencia y descripcion de la experiencia",
        "parameters": response_schema
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0.5,
    messages=messages,
    functions=functions,
    function_call={"name": "respuesta_formateada"}
)

if response.choices[0].message.function_call:
    function_call = response.choices[0].message.function_call
    structured_output = json.loads(function_call.arguments)
    print("Respuesta:\n", json.dumps(structured_output, indent=4, ensure_ascii=False))
    wrapped_description = textwrap.fill(structured_output['descripcion de la experiencia'], width=120)
    print(f"Descripción de la experiencia:\n{wrapped_description}")
else:
    print("Error:", response.choices[0].message.content)

Respuesta:
 {
    "puntuacion": 89.01,
    "experiencia": [
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor/a de puesto de mercado",
            "duracion": 5
        },
        {
            "empresa": "GASTROTEKA ORDIZIA 1990",
            "puesto": "Camarero/a de barra",
            "duracion": 6
        },
        {
            "empresa": "AGRISOLUTIONS",
            "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
            "duracion": 48
        }
    ],
    "descripcion de la experiencia": "El candidato ha acumulado una sólida experiencia en atención al cliente y manejo de operaciones de caja, especialmente a través de su trabajo en Mercadona como Vendedor/a de puesto de mercado durante 5 meses. Además, su paso por GASTROTEKA ORDIZIA 1990 como Camarero/a de barra durante 6 meses le ha permitido desarrollar habilidades interpersonales y de servicio al cliente. Por último, su experiencia de 48 meses en AGRISOLUTIONS como Auxiliar de Mantenimiento I

### 4. Prueba final del código completo

Una vez comprobado el proceso completo, podemos encapsular todo el código en la clase definida al inicio de este notebook. Finalmente, guardaremos el módulo en un fichero .py al que llamará la interfaz de usuario a diseñar en el próximo notebook.

In [ ]:
class ProcesadorCV:

    def __init__(self, api_key, cv_text, job_text, ner_pre_prompt, system_prompt, user_prompt, ner_schema, response_schema,
                inference_model="gpt-4o-mini", embeddings_model="text-embedding-3-small"):
        """
        Inicializa una instancia de la clase con los parámetros proporcionados.

        Args:
            api_key (str): La clave de API para autenticar con el cliente OpenAI.
            cv_text (str): contenido del CV en formato de texto.
            job_text (str): título de la oferta de trabajo a evaluar.
            ner_pre_prompt (str): instrucción de "reconocimiento de entidades nombradas" (NER) para el modelo en lenguaje natural.
            system_prompt (str): instrucción en lenguaje natural para la salida estructurada final.
            user_prompt (str): instrucción con los parámetros y datos calculados en el preprocesamiento.
            ner_schema (dict): esquema para la llamada con "structured outputs" al modelo de OpenAI para NER.
            response_schema (dict): esquema para la respuesta final de la aplicación.
            inference_model (str, opcional): El modelo de inferencia a utilizar. Por defecto es "gpt-4o-mini".
            embeddings_model (str, opcional): El modelo de embeddings a utilizar. Por defecto es "text-embedding-3-small".

        Atributos:
            inference_model (str): Almacena el modelo de inferencia seleccionado.
            embeddings_model (str): Almacena el modelo de embeddings seleccionado.
            client (OpenAI): Instancia del cliente OpenAI inicializada con la clave de API proporcionada.
            cv (str): Almacena el texto del currículum vitae proporcionado.

        """
        self.inference_model = inference_model
        self.embeddings_model = embeddings_model
        self.ner_pre_prompt = ner_pre_prompt
        self.user_prompt = user_prompt
        self.system_prompt = system_prompt
        self.ner_schema = ner_schema
        self.response_schema = response_schema
        self.client = OpenAI(api_key=api_key)
        self.cv = cv_text
        self.job_text = job_text
        print("Cliente inicializado como",self.client)

    def extraer_datos_cv(self, temperature=0.5):
        """
        Extrae datos estructurados de un CV con OpenAI API.
        Args:
            pre_prompt (str): instrucción para el modelo en lenguaje natural.
            schema (dict): esquema de los parámetros que se espera extraer del CV.
            temperature (float, optional): valor de temperatura para el modelo de lenguaje. Por defecto es 0.5.
        Returns:
            pd.DataFrame: DataFrame con los datos estructurados extraídos del CV.
        Raises:
            ValueError: si no se pueden extraer datos estructurados del CV.
        """
        response = self.client.chat.completions.create(
            model=self.inference_model,
            temperature=temperature,
            messages=[
                {"role": "system", "content": self.ner_pre_prompt},
                {"role": "user", "content": self.cv}
            ],
            functions=[
                {
                    "name": "extraer_datos_cv",
                    "description": "Extrae tabla con títulos de puesto de trabajo, nombres de empresa y períodos de un CV.",
                    "parameters": self.ner_schema
                }
            ],
            function_call="auto"
        )

        if response.choices[0].message.function_call:
            function_call = response.choices[0].message.function_call
            structured_output = json.loads(function_call.arguments)
            if structured_output.get("experiencia"):
                df_cv = pd.DataFrame(structured_output["experiencia"]) 
                return df_cv
            else:
                raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
        else:
            raise ValueError(f"No se han podido extraer datos estructurados: {response.choices[0].message.content}")
        

    def procesar_periodos(self, df):    
        """
        Procesa los períodos en un DataFrame y añade columnas con las fechas de inicio, fin y duración en meses. 
        Si no hay fecha de fin, se considera la fecha actual.
        Args:
            df (pandas.DataFrame): DataFrame que contiene una columna 'periodo' con períodos en formato 'YYYYMM-YYYYMM' o 'YYYYMM'.
        Returns:
            pandas.DataFrame: DataFrame con columnas adicionales 'fec_inicio', 'fec_final' y 'duracion'.
                - 'fec_inicio' (datetime.date): Fecha de inicio del período.
                - 'fec_final' (datetime.date): Fecha de fin del período.
                - 'duracion' (int): Duración del período en meses.
        """
        # Función lambda para procesar el período
        def split_periodo(periodo):
            dates = periodo.split('-')
            start_date = datetime.strptime(dates[0], "%Y%m")
            if len(dates) > 1:
                end_date = datetime.strptime(dates[1], "%Y%m")
            else:
                end_date = datetime.now()
            return start_date, end_date

        df[['fec_inicio', 'fec_final']] = df['periodo'].apply(lambda x: pd.Series(split_periodo(x)))

        # Formateamos las fechas para mostrar mes, año, y el primer día del mes (dado que el día es irrelevante y no se suele especificar)
        df['fec_inicio'] = df['fec_inicio'].dt.date
        df['fec_final'] = df['fec_final'].dt.date

        # Añadimos una columna con la duración en meses
        df['duracion'] = df.apply(
            lambda row: (row['fec_final'].year - row['fec_inicio'].year) * 12 + 
                        row['fec_final'].month - row['fec_inicio'].month, 
            axis=1
        )

        return df


    def calcular_embeddings(self, df, column='puesto', model_name='text-embedding-3-small'):
        """
        Calcula los embeddings de una columna de un dataframe con OpenAI API.
        Args:
            cv_df (pandas.DataFrame): DataFrame con los datos de los CV.
            column (str, optional): Nombre de la columna que contiene los datos a convertir en embeddings. Por defecto es 'puesto'.
            model_name (str, optional): Nombre del modelo de embeddings. Por defecto es 'text-embedding-3-small'.
        """
        df['embeddings'] = df[column].apply(
            lambda puesto: self.client.embeddings.create(
                input=puesto, 
                model=model_name
            ).data[0].embedding
        )
        return df


    def calcular_distancias(self, df, column='embeddings', model_name='text-embedding-3-small'):
        """
        Calcula la distancia coseno entre los embeddings del texto y los incluidos en una columna del dataframe.
        Params:
        df (pandas.DataFrame): DataFrame que contiene los embeddings.
        column (str, optional): nombre de la columna del DataFrame que contiene los embeddings. Por defecto, 'embeddings'.
        model_name (str, optional): modelo de embeddings de la API de OpenAI. Por defecto "text-embedding-3-small".
        Returns:
        pandas.DataFrame: DataFrame ordenado de menor a mayor distancia, con las distancias en una nueva columna.
        """
        response = self.client.embeddings.create(
            input=self.job_text,
            model=model_name
        )
        emb_compare = response.data[0].embedding

        df['distancia'] = df[column].apply(lambda emb: spatial.distance.cosine(emb, emb_compare))
        df.drop(columns=[column], inplace=True)
        df.sort_values(by='distancia', ascending=True, inplace=True)
        return df


    def calcular_puntuacion(self, df, req_experience, positions_cap=4, dist_threshold_low=0.6, dist_threshold_high=0.7):
        """
        Calcula la puntuación de un CV a partir de su tabla de distancias (con respecto a un puesto dado) y duraciones. 

        Params:
        df (pandas.DataFrame): datos de un CV incluyendo diferentes experiencias incluyendo duracies y distancia previamente calculadas sobre los embeddings de un puesto de trabajo
        req_experience (float): experiencia requerida en meses para el puesto de trabajo (valor de referencia para calcular una puntuación entre 0 y 100 en base a diferentes experiencias)
        positions_cap (int, optional): Maximum number of positions to consider for scoring. Defaults to 4.
        dist_threshold_low (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV se considera "equivalente" al de la oferta.
        max_dist_threshold (float, optional): Distancia entre embeddings a partir de la cual el puesto del CV no puntúa.
        
        Returns:
        pandas.DataFrame: DataFrame original añadiendo una columna con las puntuaciones individuales contribuidas por cada puesto.
        float: Puntuación total entre 0 y 100.
        """
        # A efectos de puntuación, computamos para cada puesto como máximo el número total de meses de experiencia requeridos
        df['duration_capped'] = df['duracion'].apply(lambda x: min(x, req_experience))
        # Normalizamos la distancia entre 0 y 1, siendo 0 la distancia mínima y 1 la máxima
        df['adjusted_distance'] = df['distancia'].apply(
            lambda x: 0 if x <= dist_threshold_low else (
                1 if x >= dist_threshold_high else (x - dist_threshold_low) / (dist_threshold_high - dist_threshold_low)
            )
        )
        # Cada puesto puntúa en base a su duración y a la inversa de la distancia (a menor distancia, mayor puntuación)
        df['position_score'] = round(((1 - df['adjusted_distance']) * (df['duration_capped']/req_experience) * 100), 2)
        # Descartamos puestos con distancia superior al umbral definido (asignamos puntuación 0), y ordenamos por puntuación
        df.loc[df['distancia'] >= dist_threshold_high, 'position_score'] = 0
        df = df.sort_values(by='position_score', ascending=False)
        # Nos quedamos con los puestos con mayor puntuación (positions_cap)
        df.iloc[positions_cap:, df.columns.get_loc('position_score')] = 0
        # Totalizamos (no debería superar 100 nunca, pero ponemos un límite para asegurar) y redondeamos a dos decimales
        total_score = round(min(df['position_score'].sum(), 100), 2)
        return df, total_score
    
    def filtra_experiencia_relevante(self, df):
        """
        Filtra las experiencias relevantes del dataframe y las devuelve en formato diccionario.
        Args:
            df (pandas.DataFrame): DataFrame con la información completa de experiencia.
        Returns:
            dict: Diccionario con las experiencias relevantes.
        """
        df_experiencia =  df[df['position_score'] > 0].copy()
        df_experiencia.drop(columns=['periodo', 'fec_inicio', 'fec_final', 
                                     'distancia', 'duration_capped', 'adjusted_distance'], inplace=True)
        experiencia_dict = df_experiencia.to_dict(orient='list')
        return experiencia_dict
    
    def llamada_final(self, req_experience, puntuacion, dict_experiencia):
        """
        Realiza la llamada final al modelo de lenguaje para generar la respuesta final.
        Args:
        req_experience (int): Experiencia requerida en meses para el puesto de trabajo.
        puntuacion (float): Puntuación total del CV.
        dict_experiencia (dict): Diccionario con las experiencias relevantes.
        Returns:
        dict: Diccionario con la respuesta final.
        """
        messages = [
            {
                "role": "system",
                "content": self.system_prompt
            },
            {
                "role": "user",
                "content": self.user_prompt.format(job=job_text, req_experience=req_experience,puntuacion=puntuacion, exp=dict_experiencia)
            }
        ]

        functions = [
            {
                "name": "respuesta_formateada",
                "description": "Devuelve el objeto con puntuacion, experiencia y descripcion de la experiencia",
                "parameters": self.response_schema
            }
        ]

        response = client.chat.completions.create(
            model=self.inference_model,
            temperature=0.5,
            messages=messages,
            functions=functions,
            function_call={"name": "respuesta_formateada"}
        )

        if response.choices[0].message.function_call:
            function_call = response.choices[0].message.function_call
            structured_output = json.loads(function_call.arguments)
            print("Respuesta:\n", json.dumps(structured_output, indent=4, ensure_ascii=False))
            wrapped_description = textwrap.fill(structured_output['descripcion de la experiencia'], width=120)
            print(f"Descripción de la experiencia:\n{wrapped_description}")
            return structured_output
        else:
            raise ValueError(f"Error. No se ha podido generar respuesta:\n {response.choices[0].message.content}")
    
    def procesar_cv_completo(self, req_experience, positions_cap, dist_threshold_low, dist_threshold_high):
        """
        Procesa un CV y calcula la puntuación final.
        Args:
            req_experience (int, optional): Experiencia requerida en meses para el puesto de trabajo.
            positions_cap (int, optional): Número máximo de puestos a considerar para la puntuación.
            dist_threshold_low (float, optional): Distancia límite para considerar un puesto equivalente.
            dist_threshold_high (float, optional): Distancia límite para considerar un puesto no relevante.
        Returns:
            pd.DataFrame: DataFrame con las puntuaciones individuales contribuidas por cada puesto.
            float: Puntuación total entre 0 y 100.
        """
        df_datos_estructurados_cv = self.extraer_datos_cv()
        df_datos_estructurados_cv = self.procesar_periodos(df_datos_estructurados_cv)
        df_con_embeddings = self.calcular_embeddings(df_datos_estructurados_cv)
        df_con_distancias = self.calcular_distancias(df_con_embeddings)
        df_puntuaciones, puntuacion = self.calcular_puntuacion(df_con_distancias,
                                                                req_experience=req_experience,
                                                                positions_cap=positions_cap,
                                                                dist_threshold_low=dist_threshold_low,
                                                                dist_threshold_high=dist_threshold_high)
        dict_experiencia = self.filtra_experiencia_relevante(df_puntuaciones)
        dict_respuesta = self.llamada_final(req_experience, puntuacion, dict_experiencia)
        return dict_respuesta

In [ ]:
# Parámetros de ejecución:
job_text = "Cajero supermercado Dia"
cv_sample_path = '../../ejemplos_cvs/cv_sample.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()
# Prompts:
with open('../prompts/ner_pre_prompt.txt', 'r', encoding='utf-8') as f:
    ner_pre_prompt = f.read()
with open('../prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()
with open('../prompts/user_prompt.txt', 'r', encoding='utf-8') as f:
    user_prompt = f.read()
# Esquemas JSON:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as f:
    ner_schema = json.load(f)
with open('../json/response_schema.json', 'r', encoding='utf-8') as f:
    response_schema = json.load(f)


procesador_cvs_prueba_final = ProcesadorCV(api_key, cv_text, job_text, ner_pre_prompt, 
                                           system_prompt, user_prompt, ner_schema, response_schema)
req_experience = 24 # Experiencia requerida en meses
positions_cap=4 # Número máximo de puestos a considerar
dist_threshold_low=0.55 # Distancia límite para considerar un puesto equivalente
dist_threshold_high=0.65 # Distancia límite para considerar un puesto no relevante
dict_respuesta = procesador_cvs_prueba_final.procesar_cv_completo(req_experience=req_experience,
                                                positions_cap=positions_cap,
                                                dist_threshold_low=dist_threshold_low,
                                                dist_threshold_high=dist_threshold_high
                                                )

Cliente inicializado como <openai.OpenAI object at 0x00000159FD143790>
Respuesta:
 {
    "puntuacion": 68.6,
    "experiencia": [
        {
            "empresa": "AGRISOLUTIONS",
            "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
            "duracion": 48
        },
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor/a de puesto de mercado",
            "duracion": 5
        },
        {
            "empresa": "GASTROTEKA ORDIZIA 1990",
            "puesto": "Camarero/a de barra",
            "duracion": 6
        },
        {
            "empresa": "Autónomo",
            "puesto": "Comercial de automoviles",
            "duracion": 1
        }
    ],
    "descripcion de la experiencia": "El candidato cuenta con una experiencia total de aproximadamente 4 años en diferentes roles, aunque su experiencia más relevante para el puesto de cajero en supermercado es limitada. Ha trabajado como vendedor en Mercadona y en un puesto de mercado, lo que le ha

Probamos con otro ejemplo:

In [53]:
# Parámetros de ejecución:
job_text = "Generative AI engineer"
cv_sample_path = '../../ejemplos_cvs/DavidGR_cv.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()
# Prompts:
with open('../prompts/ner_pre_prompt.txt', 'r', encoding='utf-8') as f:
    ner_pre_prompt = f.read()
with open('../prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()
with open('../prompts/user_prompt.txt', 'r', encoding='utf-8') as f:
    user_prompt = f.read()
# Esquemas JSON:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as f:
    ner_schema = json.load(f)
with open('../json/response_schema.json', 'r', encoding='utf-8') as f:
    response_schema = json.load(f)


procesador_cvs_prueba_final = ProcesadorCV(api_key, cv_text, job_text, ner_pre_prompt, 
                                           system_prompt, user_prompt, ner_schema, response_schema)
req_experience = 48 # Experiencia requerida en meses
positions_cap=10 # Número máximo de puestos a considerar
dist_threshold_low=0.5 # Distancia límite para considerar un puesto equivalente
dist_threshold_high=0.7 # Distancia límite para considerar un puesto no relevante
dict_respuesta = procesador_cvs_prueba_final.procesar_cv_completo(req_experience=req_experience,
                                                positions_cap=positions_cap,
                                                dist_threshold_low=dist_threshold_low,
                                                dist_threshold_high=dist_threshold_high
                                                )

Cliente inicializado como <openai.OpenAI object at 0x00000159FD143750>
Respuesta:
 {
    "puntuacion": 100,
    "experiencia": [
        {
            "empresa": "Talking to Chatbots, by Reddgr",
            "puesto": "Web Publisher and Generative AI Researcher",
            "duracion": 206
        },
        {
            "empresa": "IBM",
            "puesto": "Relationship Manager | Cognitive Solutions SaaS",
            "duracion": 43
        },
        {
            "empresa": "Acoustic",
            "puesto": "Principal Consultant | Martech SaaS",
            "duracion": 35
        },
        {
            "empresa": "IBM",
            "puesto": "Engagement Manager, in support of Acoustic | B2B SaaS Retail Analytics",
            "duracion": 10
        },
        {
            "empresa": "IBM",
            "puesto": "Engagement Manager | B2B SaaS Retail Analytics",
            "duracion": 9
        },
        {
            "empresa": "MBD Analytics",
            "puesto": "Busine